# VIX Raw  CBOE  -  Data Munging

In [1]:
from IPython.display import display_html, HTML
import pyfolio as pf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import os
import re
from time import time
import feather
from pathlib import Path

import sys
opt_path = '/Users/ebellord/PycharmProjects/opt_trade'
if  opt_path not in sys.path:
    sys.path.append(opt_path)
from ftplib import FTP

from spx_data_update import data_shop_login, UpdateSP500Data
from option_utilities import write_feather
from vix_utilities import UpdateVIXData
order_string = '/order_000008108/item_000010804/'
data_directory = UpdateSP500Data.DATA_BASE_PATH / 'CBOERawVixData'
zip_directory, csv_directory, feather_directory = [data_directory / item for item in ['zip', 'csv','feather']]

# Import raw zip files - Unpack to CSV and save as CSV

In [2]:
vx_data = UpdateVIXData()
download_raw_zip = False
if download_raw_zip:
    vx_data.download_cboe_vix()
raw_df = vx_data.raw_df
price_columns = ['open_bid', 'open_ask', 'avg_bid', 'avg_ask','close_bid',
                 'close_ask', 'open_px', 'close_px','low_px', 'high_px']                               

# Clean DataFrame

In [3]:
raw_df.loc[:, ['quote_date', 'expiry']] = raw_df.loc[:, ['quote_date', 'expiry']].apply(pd.to_datetime)
# Clean prices
last_bad_price_date = pd.to_datetime('23-March-2007') # Prior to last_bad_price_date quotes were of the type VIX*10
def divide_prices(group):
    if all(group['quote_date'] <= last_bad_price_date):
           group[price_columns] = group[price_columns] /10
    return group
clean_price_df = raw_df.groupby(['quote_date']).apply(divide_prices)

In [4]:
# Remove days with only a single traded contract
singleton_idx = clean_price_df.groupby(['quote_date']).apply(lambda x: x['quote_date'].count()==1)
list_singletons= singleton_idx[singleton_idx].index.strftime('%Y-%m-%d').tolist()
print('Number of singletons - {}'.format(len(list_singletons)))
for items in list_singletons:
    clean_price_df = clean_price_df[clean_price_df['quote_date'] != items]

#### WARNING: ELIMINATE SINGLE DATE 15-SEP-2006 #####
clean_price_df = clean_price_df[clean_price_df['quote_date'] != '15-09-2006']

Number of singletons - 1


In [5]:
# Split data into good_children where expiry matched actual_expiries and problem_children that do not
expiries = pd.DatetimeIndex(raw_df['expiry'].unique())
actual_expiries = pd.DatetimeIndex(pd.read_csv(data_directory / 'monthly_expirations.csv',
                                                  header=None,
                                                  squeeze=True))
is_not_actual_expiry = clean_price_df['expiry'].apply(lambda x: x not in actual_expiries)
problem_children = clean_price_df[is_not_actual_expiry]
good_children = clean_price_df[is_not_actual_expiry.apply(lambda x: not(x))]
problem_expiries = pd.DatetimeIndex(problem_children['expiry'].unique()).sort_values()

In [6]:
# Remove weeklies
print('Number of problem children - {}'.format(len(problem_children)))
print('Number of problem expiries - {}'.format(len(problem_expiries)))
wednesday_expiries = [pd.date_range('5 August 2015', actual_expiries[-1],
                                    freq=item) for item in ['1W-WED',
                                                            '2W-WED',
                                                            '4W-WED',
                                                            '5W-WED']]
all_wed_expiries = pd.to_datetime([])
for item in wednesday_expiries:
    all_wed_expiries = all_wed_expiries.union(item)

irregular_expiries =pd.DatetimeIndex([
'24 November 2015', #(Tuesday, due to Christmas Day on Friday)
'1 December 2015', #(Tuesday, due to New Year’s Day on Friday)
'23 February 2016', #(Tuesday, due to Good Friday)
'14 March 2017', #(Tuesday, due to Good Friday)
'27 February 2018', #(Tuesday, due to Good Friday)
'3 July 2018', #(Tuesday, due to 4 July on Wednesday)
'24 December 2019', #(Tuesday, due to Christmas Day on Wednesday, to be confirmed)
'31 December 2019']) #(Tuesday, due to New Year’s Day on Wednesday, to be confirmed)
all_wed_expiries = all_wed_expiries.union(irregular_expiries)
# remove wednesday expiries
is_not_wed_expiry = problem_children['expiry'].apply(lambda x: x not in all_wed_expiries)
problem_children = problem_children[is_not_wed_expiry]
print('Number of problem children - {}'.format(len(problem_children)))
problem_expiries = pd.DatetimeIndex(problem_children['expiry'].unique()).sort_values()
problem_expiries.sort_values()
values = []
# Create dictionnary of missmatched expiries (keys) and actual expiries (values)
for item in problem_expiries:
    idx = np.abs(item - actual_expiries).argmin()
    values.append(actual_expiries[idx])
expiry_dict = {}
keys = problem_expiries
for key, value in zip(keys, values):
    expiry_dict[key] = value
print('Number of problem expiries - {}'.format(len(expiry_dict)))

Number of problem children - 7054
Number of problem expiries - 190
Number of problem children - 2534
Number of problem expiries - 41


In [7]:
# Dictionary for missing expiries 
missing_dict = {}
missing_key = []
missing_value = []
# Dictionary for double expiries 
double_dict = {}
double_key = []
double_value = []

for key, value in expiry_dict.items():
    validation_df = clean_price_df[clean_price_df['expiry']==expiry_dict[key]]
    if validation_df.empty:
        missing_key.append(key)
        missing_value.append(value)
    else:
        double_key.append(key)
        double_value.append(value)
        
for key, value in zip(missing_key, missing_value):
    missing_dict[key] = value

for key, value in zip(double_key, double_value):
    double_dict[key] = value
    
problem_children = problem_children.replace({'expiry': missing_dict})
# Second replacement could be combined with previous replacement however, I choose to split it 
# into two operations to underline the difference betwee the two replacements
problem_children = problem_children.replace({'expiry': double_dict})
# Remove single observation where expiry date is equal to quote date
index_same_expiry_quote_date = problem_children[problem_children['expiry'] == problem_children['quote_date']].index
problem_children_reformed = problem_children.drop(index_same_expiry_quote_date)

clean_monthly_future = pd.concat([good_children, problem_children_reformed], axis=0, ignore_index=True)
# Make sure clean_monthly_future only has actual expiries
assert(pd.DatetimeIndex(clean_monthly_future['expiry'].unique()).difference(actual_expiries).empty)
clean_monthly_future['days_left'] = clean_monthly_future['expiry'] - clean_monthly_future['quote_date']

# Mannual routine to check last problem children
# num = 8
# display(foobar[foobar['expiry']==list(double_dict.keys())[num]].sort_values('quote_date'))

# display(clean_price_df[clean_price_df['expiry']==list(double_dict.values())[num]].sort_values('quote_date').style)

# Stack futures by quote date

In [8]:
def squeeze_group(group):
    column_names  = price_columns + ['expiry', 'days_left']
    tmp_list = []
    count = 1
    one_day = pd.timedelta_range(start='1 day', periods=1)
    for item in group['days_left'].sort_values():
        if item >= one_day: # remove zero days to expiry
            n_expiry_data = group[group['days_left']==item]
            n_expiry_data = n_expiry_data[price_columns + ['expiry','days_left']]
            n_expiry_data = n_expiry_data.stack(dropna=False)
            n_expiry_data.index = n_expiry_data.index.droplevel()
            n_expiry_data = n_expiry_data.fillna({'open_px':n_expiry_data[['open_bid', 'open_ask']].mean(),
                                                'close_px':n_expiry_data[['close_bid', 'close_ask']].mean()})
            n_expiry_data['days_left'] = n_expiry_data['days_left'].days
            n_expiry_data.index = [str(col) + str(count) for col in n_expiry_data.index]
            count = count + 1
            tmp_list.append(n_expiry_data)
    stacked_df = pd.concat(tmp_list, ignore_index=False, axis=0)
    return stacked_df
%time stacked_futures = clean_monthly_future.groupby('quote_date').apply(squeeze_group)
vix_data = stacked_futures.unstack(level=1)
vix_data = vix_data.reset_index()



# Tidy up and save:
 - Creating dataframe of vix numbered futures creates a problem when there is missing data. The squeeze_group function applied to each quote_date filters numbered futures based on days left to expiry. When there is missing data a contract can end up being the nth contract multiple times. Following routine eliminates this problem by checking the number of times a contract is used for each expiry and deleting all but the last occurence.
 - Some  expiry dates do not match other expiry calendars  [Vix Expirations](https://www.macroption.com/vix-expiration-calendar/#exceptions) (DONE)
     
 First weekly expirations are 5 August 2015

In [38]:
def repeated_timestamps(vix_data, exp='expiry2'):
    df = vix_data[exp].to_frame()
    df['counter'] = (df[exp] != df[exp].shift()).mask(df[exp].isnull()).groupby(df[exp]).cumsum()
    timestamps = df.groupby(exp).apply(max)[df.groupby(exp).apply(max) > 1].dropna()
    return timestamps
matching = [s for s in vix_data.columns if "expiry" in s]
matching.remove('expiry1')
for item in matching:
    repeat_timestamps = repeated_timestamps(vix_data, exp=item)
    digit = re.findall("\d+", item)[0]
    match_columns = [s for s in vix_data.columns if digit in s]
    for time_stamp in repeat_timestamps[item]:
        bar = vix_data[vix_data[item] == time_stamp].index # index of expiries
        max_index = max([i for i, x in enumerate(np.diff(bar) > 1) if x])
        vix_data.loc[bar[:max_index + 1], match_columns] = np.nan
write_feather(vix_data, feather_directory / 'vix_data.feather')
